# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.129405e+02     1.873556e+00
 * time: 0.07543683052062988
     1     1.088428e+01     9.690852e-01
 * time: 1.2689988613128662
     2    -1.204808e+01     1.123640e+00
 * time: 1.413450002670288
     3    -3.383400e+01     8.396793e-01
 * time: 1.5494208335876465
     4    -4.745241e+01     5.843401e-01
 * time: 1.6864688396453857
     5    -5.699554e+01     2.117739e-01
 * time: 1.8364198207855225
     6    -5.991561e+01     1.142292e-01
 * time: 1.928278923034668
     7    -6.095767e+01     4.860779e-02
 * time: 2.0208139419555664
     8    -6.133965e+01     6.035881e-02
 * time: 2.113348960876465
     9    -6.161751e+01     3.716844e-02
 * time: 2.205894947052002
    10    -6.182541e+01     2.779192e-02
 * time: 2.3116648197174072
    11    -6.197538e+01     2.035466e-02
 * time: 2.404979944229126
    12    -6.203259e+01     1.915628e-02
 * time: 2.4980859756469727
    13    -6.209321e+01     1.529347e-02
 * time: 2.59037494659423

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557625
    AtomicNonlocal      14.8522599
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485365 
    Xc                  -19.3336819

    total               -62.261666458385
